In [2]:
pip install pyupbit

     -------------------------------------- 124.7/124.7 kB 7.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyupbit as ub
import ccxt
import pandas as pd
import datetime
import time
import json

In [15]:
def get_crypto_data(tickers, interval="day", count=2000):
    """
    tickers: 암호화폐의 ticker 목록 (예: ["KRW-BTC", "KRW-ETH"])
    interval: 데이터 간격 ("minute1", "minute3", "minute5", "minute10", "minute15", "minute30", "minute60", "minute240", "day", "week", "month")
    count: 가져올 데이터의 갯수 (최대 200)
    
    return: ticker를 key로 하고, 해당 데이터를 value로 하는 dictionary 반환
    """
    data = {}
    for ticker in tickers:
        ohlcv = ub.get_ohlcv(ticker, interval=interval, count=count)
        data[ticker] = ohlcv
    return data

# 예시 사용법
tickers = ub.get_tickers()

crypto_data = get_crypto_data(tickers)
for ticker, ohlcv in crypto_data.items():
    print(ticker)
    print(ohlcv.head())  # 상위 5개 데이터 출력

KRW-BTC
                           open        high         low       close  \
2018-03-02 09:00:00  12214000.0  12581000.0  12123000.0  12465000.0   
2018-03-03 09:00:00  12465000.0  12831000.0  12347000.0  12831000.0   
2018-03-04 09:00:00  12831000.0  12840000.0  12255000.0  12592000.0   
2018-03-05 09:00:00  12592000.0  12899000.0  12451000.0  12746000.0   
2018-03-06 09:00:00  12745000.0  12746000.0  11900000.0  12023000.0   

                           volume         value  
2018-03-02 09:00:00  23945.117557  2.956371e+11  
2018-03-03 09:00:00  18163.175191  2.289656e+11  
2018-03-04 09:00:00  16125.743255  2.013402e+11  
2018-03-05 09:00:00  13919.500362  1.757026e+11  
2018-03-06 09:00:00  15007.199784  1.853153e+11  
KRW-ETH
                         open      high       low     close        volume  \
2018-03-02 09:00:00  973900.0  979800.0  959700.0  965000.0  17451.831318   
2018-03-03 09:00:00  965000.0  968500.0  951100.0  956300.0  15285.995637   
2018-03-04 09:00:00  95650

In [1]:
type(crypto_data)

NameError: name 'crypto_data' is not defined

In [21]:
def save_to_json(data, filename):
    """
    data: 저장하려는 딕셔너리 데이터. 이 데이터 안에 DataFrame 객체가 포함될 수 있음.
    filename: 저장하려는 파일의 이름 (예: "data.json")
    """
    # DataFrame 객체를 JSON 문자열로 변환
    serialized_data = {}
    for key, value in data.items():
        if isinstance(value, pd.DataFrame):
            serialized_data[key] = value.to_json(orient='split')
        else:
            serialized_data[key] = value
            
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(serialized_data, f, ensure_ascii=False, indent=4)

In [22]:
save_to_json(crypto_data, "crypto_data.json")

In [4]:
def load_from_json(filename):
    """
    filename: 불러오려는 파일의 이름 (예: "data.json")
    
    return: JSON 파일의 내용을 딕셔너리 형태로 반환. 내부에 DataFrame 문자열이 있을 경우 DataFrame으로 변환하여 반환.
    """
    with open(filename, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # JSON 문자열을 DataFrame으로 변환
    for key, value in data.items():
        if isinstance(value, str) and "index" in value and "data" in value and "columns" in value:
            data[key] = pd.read_json(value, orient='split')
    
    return data


In [24]:
loaded_data = load_from_json("crypto_data.json")
for key, value in loaded_data.items():
    print(key)
    if isinstance(value, pd.DataFrame):
        print(value.head())  # DataFrame의 경우 상위 5개 데이터 출력
    else:
        print(value)

KRW-BTC
                         open      high       low     close        volume  \
2018-03-02 09:00:00  12214000  12581000  12123000  12465000  23945.117557   
2018-03-03 09:00:00  12465000  12831000  12347000  12831000  18163.175191   
2018-03-04 09:00:00  12831000  12840000  12255000  12592000  16125.743255   
2018-03-05 09:00:00  12592000  12899000  12451000  12746000  13919.500362   
2018-03-06 09:00:00  12745000  12746000  11900000  12023000  15007.199784   

                            value  
2018-03-02 09:00:00  2.956371e+11  
2018-03-03 09:00:00  2.289656e+11  
2018-03-04 09:00:00  2.013402e+11  
2018-03-05 09:00:00  1.757026e+11  
2018-03-06 09:00:00  1.853153e+11  
KRW-ETH
                       open    high     low   close        volume  \
2018-03-02 09:00:00  973900  979800  959700  965000  17451.831318   
2018-03-03 09:00:00  965000  968500  951100  956300  15285.995637   
2018-03-04 09:00:00  956500  958000  927300  947900  15385.445487   
2018-03-05 09:00:00  947600  

In [27]:
len(loaded_data.keys())

304

In [14]:
def add_index_as_date_column(data):
    """
    data: 입력 딕셔너리. 이 딕셔너리의 각 값은 DataFrame이어야 함.
    return: 각 DataFrame에 인덱스를 "date"라는 새로운 열로 추가한 딕셔너리.
    """
    for key, value in data.items():
        if isinstance(value, pd.DataFrame):
            value['date'] = value.index
            data[key] = value
    return data

In [15]:
result_data = add_index_as_date_column(loaded_data)
for key, value in result_data.items():
    print(key)
    print(value)

KRW-BTC
                         open      high       low     close        volume  \
2018-03-02 09:00:00  12214000  12581000  12123000  12465000  23945.117557   
2018-03-03 09:00:00  12465000  12831000  12347000  12831000  18163.175191   
2018-03-04 09:00:00  12831000  12840000  12255000  12592000  16125.743255   
2018-03-05 09:00:00  12592000  12899000  12451000  12746000  13919.500362   
2018-03-06 09:00:00  12745000  12746000  11900000  12023000  15007.199784   
...                       ...       ...       ...       ...           ...   
2023-08-18 09:00:00  36588000  36848000  35384000  35890000   9256.752346   
2023-08-19 09:00:00  35899000  36240000  35587000  36056000   2670.316058   
2023-08-20 09:00:00  36071000  36225000  35754000  35949000   1654.936938   
2023-08-21 09:00:00  35946000  35967000  35500000  35769000   2142.999606   
2023-08-22 09:00:00  35769000  35877000  35605000  35671000    888.751729   

                            value                date  
2018-03-02 

In [17]:
result_data

{'KRW-BTC':                          open      high       low     close        volume  \
 2018-03-02 09:00:00  12214000  12581000  12123000  12465000  23945.117557   
 2018-03-03 09:00:00  12465000  12831000  12347000  12831000  18163.175191   
 2018-03-04 09:00:00  12831000  12840000  12255000  12592000  16125.743255   
 2018-03-05 09:00:00  12592000  12899000  12451000  12746000  13919.500362   
 2018-03-06 09:00:00  12745000  12746000  11900000  12023000  15007.199784   
 ...                       ...       ...       ...       ...           ...   
 2023-08-18 09:00:00  36588000  36848000  35384000  35890000   9256.752346   
 2023-08-19 09:00:00  35899000  36240000  35587000  36056000   2670.316058   
 2023-08-20 09:00:00  36071000  36225000  35754000  35949000   1654.936938   
 2023-08-21 09:00:00  35946000  35967000  35500000  35769000   2142.999606   
 2023-08-22 09:00:00  35769000  35877000  35605000  35671000    888.751729   
 
                             value                d

In [18]:
def dict_to_dataframe(data, key_column_name="key"):
    """
    data: 변환하려는 딕셔너리. 이 딕셔너리의 각 값은 DataFrame이어야 함.
    key_column_name: 딕셔너리의 키 값을 저장할 열의 이름.
    
    return: 딕셔너리를 변환한 DataFrame. 딕셔너리의 각 키 값은 새로운 열로 추가됨.
    """
    dfs = []
    for key, value in data.items():
        if isinstance(value, pd.DataFrame):
            df = value.copy()
            df[key_column_name] = key
            dfs.append(df)
    
    return pd.concat(dfs, ignore_index=True)

In [19]:
result_df = dict_to_dataframe(result_data, key_column_name="currency")
print(result_df)

                open          high           low         close        volume  \
0       1.221400e+07  1.258100e+07  1.212300e+07  1.246500e+07  2.394512e+04   
1       1.246500e+07  1.283100e+07  1.234700e+07  1.283100e+07  1.816318e+04   
2       1.283100e+07  1.284000e+07  1.225500e+07  1.259200e+07  1.612574e+04   
3       1.259200e+07  1.289900e+07  1.245100e+07  1.274600e+07  1.391950e+04   
4       1.274500e+07  1.274600e+07  1.190000e+07  1.202300e+07  1.500720e+04   
...              ...           ...           ...           ...           ...   
388178  2.300000e+02  2.360000e+02  2.140000e+02  2.220000e+02  3.526380e+08   
388179  2.220000e+02  2.240000e+02  2.030000e+02  2.090000e+02  2.692290e+08   
388180  2.090000e+02  2.150000e+02  1.950000e+02  2.050000e+02  2.662203e+08   
388181  2.050000e+02  2.140000e+02  1.990000e+02  2.040000e+02  1.685148e+08   
388182  1.799800e-04  1.899700e-04  1.709500e-04  1.821400e-04  1.900387e+06   

               value                dat

In [20]:
result_df = result_df[['date', 'open', 'high', 'low', 'close', 'volume', 'value', 'currency']]
result_df

,date,open,high,low,close,volume,value,currency
0,2018-03-02 09:00:00,1.221400e+07,1.258100e+07,1.212300e+07,1.246500e+07,2.394512e+04,2.956371e+11,KRW-BTC
1,2018-03-03 09:00:00,1.246500e+07,1.283100e+07,1.234700e+07,1.283100e+07,1.816318e+04,2.289656e+11,KRW-BTC
2,2018-03-04 09:00:00,1.283100e+07,1.284000e+07,1.225500e+07,1.259200e+07,1.612574e+04,2.013402e+11,KRW-BTC
3,2018-03-05 09:00:00,1.259200e+07,1.289900e+07,1.245100e+07,1.274600e+07,1.391950e+04,1.757026e+11,KRW-BTC
4,2018-03-06 09:00:00,1.274500e+07,1.274600e+07,1.190000e+07,1.202300e+07,1.500720e+04,1.853153e+11,KRW-BTC
...,...,...,...,...,...,...,...,...
388178,2023-08-19 09:00:00,2.300000e+02,2.360000e+02,2.140000e+02,2.220000e+02,3.526380e+08,7.878727e+10,KRW-SEI
388179,2023-08-20 09:00:00,2.220000e+02,2.240000e+02,2.030000e+02,2.090000e+02,2.692290e+08,5.702601e+10,KRW-SEI
388180,2023-08-21 09:00:00,2.090000e+02,2.150000e+02,1.950000e+02,2.050000e+02,2.662203e+08,5.395445e+10,KRW-SEI
388181,2023-08-22 09:00:00,2.050000e+02,2.140000e+02,1.990000e+02,2.040000e+02,1.685148e+08,3.463835e+10,KRW-SEI


In [23]:
result_df.to_csv("all_cryptos_krw_pyupbit.csv", index=False)